In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

# Function to get HTML response
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None
    
# Function to parse fight details from an event page
def parse_fight_details(html, event_name, event_date, event_location):
    soup = BeautifulSoup(html, 'html.parser')
    fight_details = []
    fighter_urls = {}

    for fight_row in soup.find_all('tr', class_='b-fight-details__table-row'):
        details = {}

        fighters = fight_row.find_all('a', class_='b-link_style_black')
        
        if len(fighters) == 2:
            details['f1'] = fighters[0].text.strip()
            details['f2'] = fighters[1].text.strip()
            fighter_urls[details['f1']] = fighters[0].get('href')
            fighter_urls[details['f2']] = fighters[1].get('href')

        details['card_name'] = event_name
        details['fight_date'] = event_date
        details['fights_location'] = event_location
        details['fight_url'] = fight_row.get('data-link')
        fight_details.append(details)

    return fight_details[1:], fighter_urls

# Read event URLs from CSV file
df = pd.read_csv('ufc_fight_nights.csv')

all_fights = []
all_fighter_urls = {}

for index, row in df.iterrows():
    fight_night_url = row['url']
    event_html = get_html(fight_night_url)
    if event_html:
        fights, fighter_urls = parse_fight_details(event_html, row['event_name'], row['date'], row['location'])
        all_fights.extend(fights)
        all_fighter_urls.update(fighter_urls)

In [4]:
# Create DataFrame and save to CSV
fight_df = pd.DataFrame(all_fights)
fight_df.to_csv('ufc_fight_data.csv', index=False)
# From dictionary 
all_fighter_urls = all_fighter_urls.values()
all_fighter_urls = pd.DataFrame(all_fighter_urls)
all_fighter_urls.to_csv('all_fighter_url.csv', index=False)